# Smartphone Recommendation System


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

## Data Loading and Preparation

In [2]:
df_data = pd.read_csv(r"..\DATA\cellphones data.csv")
df_ratings = pd.read_csv(r"..\DATA\cellphones ratings.csv")
df_users = pd.read_csv(r"..\DATA\cellphones users.csv")

In [3]:
df = df_data.merge(df_ratings).merge(df_users)

In [4]:
df.head()

,cellphone_id,brand,model,operating system,internal memory,RAM,performance,main camera,selfie camera,battery size,screen size,weight,price,release date,user_id,rating,age,gender,occupation
0,0,Apple,iPhone SE (2022),iOS,128,4,7.23,12,7,2018,4.7,144,429,18/03/2022,8,5,25,Female,Manager
1,0,Apple,iPhone SE (2022),iOS,128,4,7.23,12,7,2018,4.7,144,429,18/03/2022,16,1,31,Female,sales
2,0,Apple,iPhone SE (2022),iOS,128,4,7.23,12,7,2018,4.7,144,429,18/03/2022,30,4,34,Male,ICT Officer
3,0,Apple,iPhone SE (2022),iOS,128,4,7.23,12,7,2018,4.7,144,429,18/03/2022,35,6,41,Male,self employed
4,0,Apple,iPhone SE (2022),iOS,128,4,7.23,12,7,2018,4.7,144,429,18/03/2022,52,5,25,Male,software developer


In [5]:
df['rating'].unique()

array([ 5,  1,  4,  6, 10,  8,  7,  9,  2,  3, 18], dtype=int64)

In [6]:
df['model'].unique()

array(['iPhone SE (2022)', 'iPhone 13 Mini', 'iPhone 13', 'iPhone 13 Pro',
       'iPhone 13 Pro Max', 'iPhone XR', 'Zenfone 8', 'Galaxy A13',
       'Galaxy A32', 'Galaxy A53', 'Galaxy S22', 'Galaxy S22 Plus',
       'Galaxy S22 Ultra', 'Galaxy Z Flip 3', 'Galaxy Z Fold 3',
       'Pixel 6 \xa0', 'Pixel 6a', 'Pixel 6 Pro\xa0', 'Nord N20',
       'Nord 2T', '10 Pro', '10T', 'Find X5 Pro', 'X80 Pro',
       'Redmi Note 11', '11T Pro', '12 Pro', 'Poco F4', 'Xperia Pro',
       'Moto G Stylus (2022)', 'Moto G Play (2021)', 'Moto G Pure',
       'Moto G Power (2022)'], dtype=object)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cellphone_id      990 non-null    int64  
 1   brand             990 non-null    object 
 2   model             990 non-null    object 
 3   operating system  990 non-null    object 
 4   internal memory   990 non-null    int64  
 5   RAM               990 non-null    int64  
 6   performance       990 non-null    float64
 7   main camera       990 non-null    int64  
 8   selfie camera     990 non-null    int64  
 9   battery size      990 non-null    int64  
 10  screen size       990 non-null    float64
 11  weight            990 non-null    int64  
 12  price             990 non-null    int64  
 13  release date      990 non-null    object 
 14  user_id           990 non-null    int64  
 15  rating            990 non-null    int64  
 16  age               990 non-null    int64  
 1

## Feature Engineering

In [8]:
X = pd.get_dummies(df.drop(['release date', 'brand', 'model', 'cellphone_id', 'user_id '], axis=1), drop_first=True, dtype=int)
y = df['model']

KeyError: "['user_id '] not found in axis"

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

In [ ]:
len(X.columns)

## Model Training

In [ ]:
list_err = []
for i in range(1, 40):
    ada_model = AdaBoostClassifier(n_estimators=i)
    ada_model.fit(X_train, y_train)
    y_pred = ada_model.predict(X_test)
    y_error = 1 - accuracy_score(y_test, y_pred)

    list_err.append(y_error)

#### Plot error rates

In [ ]:
plt.plot(range(1, 40), list_err)
plt.xlabel('Number of estimators')
plt.ylabel('Error rate')
plt.savefig('../Images/plot_error_rates.png')

In [ ]:
optimal_n = np.argmin(list_err) + 1
ada_model = AdaBoostClassifier(n_estimators=optimal_n)
ada_model.fit(X_train, y_train)

## Feature Importance Analysis

In [ ]:
ada_model.feature_importances_

In [ ]:
data_analyse = pd.DataFrame(index=X.columns, 
                           data=ada_model.feature_importances_, 
                           columns=['Importance'])

In [ ]:
data_analyse

In [ ]:
data_analyse_import = data_analyse[data_analyse['Importance'] > 0].sort_values('Importance')

In [ ]:
data_analyse_import

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=data_analyse_import, 
            x=data_analyse_import.index, 
            y=data_analyse_import['Importance'], 
            palette='viridis')
plt.xticks(rotation=90)
plt.title('Feature Importance for Cellphone Model Recommendation')
plt.tight_layout()
plt.savefig('../Images/feature_importance.png')